# A first look at the data

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

Open questions:
* How do I make sure that there are not two people who have the same name?
    * Partei, Beruf, Ort, Fraktion, etc sind alle nicht eindeutig. Geb_jahr könte helfen, aber ist nicht für alle vorhanden...

### 1. Simple metrics for the network

**Load the data**

In [4]:
file_abg = Path('../Daten_Thurgau/Mitglieder_GR_2008_2021.csv')
file_vor = Path('../Daten_Thurgau/Vorstoesse_GR_ab_2012_Themen_def.csv')

In [5]:
vor = pd.read_csv(file_vor,delimiter=';')
abg = pd.read_csv(file_abg,delimiter=';')

In [6]:
abg_2012 = abg.loc[abg['Jahr']>=2012]
abg_list = pd.unique(abg_2012['Vorname'] + ' ' + abg_2012['Name'])
print(f'Wir haben {len(abg_list)} unabhängige Abgeordnete seit 2012') 

Wir haben 275 unabhängige Abgeordnete seit 2012


**Selecting a subset of Vorstösse**

In [7]:
vor_use=vor
# vor_use=vor[vor['VORSTOSSART_CODE'].isin([1,2,3,7])]

In [8]:
vor_abg = vor_use[[f'VORSTOESSER_IN_{i}' for i in range(1,11)]]
netzwerk = pd.DataFrame(columns=['Name','Colab','Vorst'])
for name in abg_list: # loop over the abgeordneten
    name_vorstoesse = [] # stores vorstoss indexes where this person was part of
    name_colab = [] # stores names of colaborators
    for i in vor_use.index: # loop over the vorstoesse
        unterz = list(vor_abg.loc[i].dropna())
        if name in unterz: # check if this person has supported the vorstoss
            name_vorstoesse.append(i)
            for u in unterz: # loop over unterzeichner
                if u not in name_colab and u != name: name_colab.append((u))
    netzwerk.loc[netzwerk.shape[0]] = [name,name_colab,name_vorstoesse]

In [ ]:
# Form: 
vor_abg = vor_use[[f'VORSTOESSER_IN_{i}' for i in range(1,11)]]
netzwerk = pd.DataFrame(columns=['Name','Colab','Vorst'])
for name in abg_list: # loop over the abgeordneten
    name_vorstoesse = [] # stores vorstoss indexes where this person was part of
    name_colab = [] # stores names of colaborators
    for i in vor_use.index: # loop over the vorstoesse
        unterz = list(vor_abg.loc[i].dropna())
        if name in unterz: # check if this person has supported the vorstoss
            name_vorstoesse.append(i)
            for u in unterz: # loop over unterzeichner
                if u not in name_colab and u != name: name_colab.append((u))
    netzwerk.loc[netzwerk.shape[0]] = [name,name_colab,name_vorstoesse]

In [112]:
netzwerk


,Name,Colab,Vorst
0,Inge Abegglen,"[Alex Frei, Hans Munz, Helen Jordi, Erwin Imhof]",[136]
1,Wolfgang Ackerknecht,"[Peter Bühler, Max Möckli, Daniel Vetterli, R...","[138, 273, 421, 428, 430, 535, 538, 606, 711]"
2,Elsbeth Aepli Stettler,"[Christian Koch, Marlies Näf, Walter Schönholz...","[39, 52]"
3,Margrit Aerne,"[Roland A. Huber, Cäcilia Bosshard, Turi Schal...","[116, 462]"
4,Clemens Albrecht,[],[]
...,...,...,...
270,David Zimmermann,"[Hans Munz, Hanspeter Gantenbein, Josef Gemper...","[127, 402, 424, 467, 478, 491, 658, 699, 958, ..."
271,Andreas Zuber,[Ruedi Zbinden],[983]
272,Käthi Zürcher,[],[]
273,Felix Züst,[],[]


**Check what is the mean, median, min, max connectivity**

In [114]:
# Full network
print('Mean connectivity:', netzwerk['Colab'].apply(len).mean())
print('Median connectivity:', netzwerk['Colab'].apply(len).median())
print('Max connectivity:', netzwerk['Colab'].apply(len).max())

Mean connectivity: 12.04
Median connectivity: 6.0
Max connectivity: 71


In [26]:
# Vorstösse 1,2,3,7 (without simple requests)
print('Mean connectivity:', netzwerk['Colab'].apply(len).mean())
print('Median connectivity:', netzwerk['Colab'].apply(len).median())
print('Max connectivity:', netzwerk['Colab'].apply(len).max())

Mean connectivity: 6.807272727272728
Median connectivity: 3.0
Max connectivity: 43


In [27]:
netzwerk.loc[netzwerk['Colab'].apply(len).idxmax()]

Name                                        Josef Gemperle
Colab    [Toni Kappeler, Bernhard Wälti, Thomas Böhni, ...
Vorst    [5, 12, 25, 32, 74, 86, 87, 105, 213, 240, 403...
Name: 64, dtype: object

Der Abgeornete mit den meisten Zusammenarbeiten ist Josef Gemperle von der CVP.

**Look at how often people collaborate**